# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 16 2022 12:54PM,239772,10,Enova-10147,Emerginnova,Released
1,Jun 16 2022 12:54PM,239772,10,2-100448-2,Emerginnova,Released
2,Jun 16 2022 12:48PM,239771,10,8144204-F,"E5 Pharma, LLC",Released
3,Jun 16 2022 12:48PM,239771,10,8144205-F,"E5 Pharma, LLC",Released
4,Jun 16 2022 12:48PM,239771,10,8144206-F,"E5 Pharma, LLC",Released
5,Jun 16 2022 12:48PM,239771,10,8144207-F,"E5 Pharma, LLC",Released
6,Jun 16 2022 12:48PM,239771,10,8144208-F,"E5 Pharma, LLC",Released
7,Jun 16 2022 12:48PM,239771,10,8144209-F,"E5 Pharma, LLC",Released
8,Jun 16 2022 12:48PM,239771,10,8144210-F,"E5 Pharma, LLC",Released
9,Jun 16 2022 12:48PM,239771,10,8144211-F,"E5 Pharma, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
44,239768,Released,2
45,239769,Released,2
46,239770,Released,34
47,239771,Released,12
48,239772,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239768,NaN,NaN,2.0
239769,NaN,NaN,2.0
239770,NaN,NaN,34.0
239771,NaN,NaN,12.0
239772,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239654,31.0,42.0,1.0
239665,0.0,1.0,0.0
239672,0.0,0.0,29.0
239679,11.0,1.0,2.0
239683,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239654,31,42,1
239665,0,1,0
239672,0,0,29
239679,11,1,2
239683,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239654,31,42,1
1,239665,0,1,0
2,239672,0,0,29
3,239679,11,1,2
4,239683,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239654,31,42,1
1,239665,,1,
2,239672,,,29
3,239679,11,1,2
4,239683,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 16 2022 12:54PM,239772,10,Emerginnova
2,Jun 16 2022 12:48PM,239771,10,"E5 Pharma, LLC"
14,Jun 16 2022 12:40PM,239770,10,ISDIN Corporation
48,Jun 16 2022 12:38PM,239769,10,Bio-PRF
50,Jun 16 2022 12:35PM,239768,10,"Methapharm, Inc."
52,Jun 16 2022 12:27PM,239767,15,"Virtus Pharmaceuticals, LLC"
65,Jun 16 2022 12:24PM,239765,15,"Person & Covey, Inc."
86,Jun 16 2022 12:22PM,239766,16,ACG North America LLC
87,Jun 16 2022 11:58AM,239763,16,Sartorius Stedim North America
88,Jun 16 2022 11:57AM,239762,16,Sartorius Stedim North America


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 16 2022 12:54PM,239772,10,Emerginnova,,,2
1,Jun 16 2022 12:48PM,239771,10,"E5 Pharma, LLC",,,12
2,Jun 16 2022 12:40PM,239770,10,ISDIN Corporation,,,34
3,Jun 16 2022 12:38PM,239769,10,Bio-PRF,,,2
4,Jun 16 2022 12:35PM,239768,10,"Methapharm, Inc.",,,2
5,Jun 16 2022 12:27PM,239767,15,"Virtus Pharmaceuticals, LLC",,,13
6,Jun 16 2022 12:24PM,239765,15,"Person & Covey, Inc.",,7,14
7,Jun 16 2022 12:22PM,239766,16,ACG North America LLC,,1,
8,Jun 16 2022 11:58AM,239763,16,Sartorius Stedim North America,,,1
9,Jun 16 2022 11:57AM,239762,16,Sartorius Stedim North America,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 12:54PM,239772,10,Emerginnova,2,,
1,Jun 16 2022 12:48PM,239771,10,"E5 Pharma, LLC",12,,
2,Jun 16 2022 12:40PM,239770,10,ISDIN Corporation,34,,
3,Jun 16 2022 12:38PM,239769,10,Bio-PRF,2,,
4,Jun 16 2022 12:35PM,239768,10,"Methapharm, Inc.",2,,
5,Jun 16 2022 12:27PM,239767,15,"Virtus Pharmaceuticals, LLC",13,,
6,Jun 16 2022 12:24PM,239765,15,"Person & Covey, Inc.",14,7,
7,Jun 16 2022 12:22PM,239766,16,ACG North America LLC,,1,
8,Jun 16 2022 11:58AM,239763,16,Sartorius Stedim North America,1,,
9,Jun 16 2022 11:57AM,239762,16,Sartorius Stedim North America,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 12:54PM,239772,10,Emerginnova,2,,
1,Jun 16 2022 12:48PM,239771,10,"E5 Pharma, LLC",12,,
2,Jun 16 2022 12:40PM,239770,10,ISDIN Corporation,34,,
3,Jun 16 2022 12:38PM,239769,10,Bio-PRF,2,,
4,Jun 16 2022 12:35PM,239768,10,"Methapharm, Inc.",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 12:54PM,239772,10,Emerginnova,2.0,NaN,NaN
1,Jun 16 2022 12:48PM,239771,10,"E5 Pharma, LLC",12.0,NaN,NaN
2,Jun 16 2022 12:40PM,239770,10,ISDIN Corporation,34.0,NaN,NaN
3,Jun 16 2022 12:38PM,239769,10,Bio-PRF,2.0,NaN,NaN
4,Jun 16 2022 12:35PM,239768,10,"Methapharm, Inc.",2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 16 2022 12:54PM,239772,10,Emerginnova,2.0,0.0,0.0
1,Jun 16 2022 12:48PM,239771,10,"E5 Pharma, LLC",12.0,0.0,0.0
2,Jun 16 2022 12:40PM,239770,10,ISDIN Corporation,34.0,0.0,0.0
3,Jun 16 2022 12:38PM,239769,10,Bio-PRF,2.0,0.0,0.0
4,Jun 16 2022 12:35PM,239768,10,"Methapharm, Inc.",2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2637182,84.0,7.0,0.0
12,479495,2.0,1.0,0.0
15,1198810,48.0,8.0,0.0
16,1438499,5.0,1.0,0.0
18,239686,1.0,0.0,0.0
19,479431,3.0,1.0,11.0
20,2157270,49.0,62.0,31.0
22,479449,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2637182,84.0,7.0,0.0
1,12,479495,2.0,1.0,0.0
2,15,1198810,48.0,8.0,0.0
3,16,1438499,5.0,1.0,0.0
4,18,239686,1.0,0.0,0.0
5,19,479431,3.0,1.0,11.0
6,20,2157270,49.0,62.0,31.0
7,22,479449,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,84.0,7.0,0.0
1,12,2.0,1.0,0.0
2,15,48.0,8.0,0.0
3,16,5.0,1.0,0.0
4,18,1.0,0.0,0.0
5,19,3.0,1.0,11.0
6,20,49.0,62.0,31.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,84.0
1,12,Released,2.0
2,15,Released,48.0
3,16,Released,5.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,22
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,11.0,31.0,0.0
Executing,7.0,1.0,8.0,1.0,0.0,1.0,62.0,0.0
Released,84.0,2.0,48.0,5.0,1.0,3.0,49.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,22
0,Completed,0.0,0.0,0.0,0.0,0.0,11.0,31.0,0.0
1,Executing,7.0,1.0,8.0,1.0,0.0,1.0,62.0,0.0
2,Released,84.0,2.0,48.0,5.0,1.0,3.0,49.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,22
0,Completed,0.0,0.0,0.0,0.0,0.0,11.0,31.0,0.0
1,Executing,7.0,1.0,8.0,1.0,0.0,1.0,62.0,0.0
2,Released,84.0,2.0,48.0,5.0,1.0,3.0,49.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()